### Imports and model specifications

In [9]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

gr.connect_all_nodes([bob, alice, charlie, dan])

In [10]:
# Model Owner
# Support fetch plan + AST tensor
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__(id="convnet")
        self.conv1 = nn.Conv2d(3, 4, 5, 1)
        self.fc1 = nn.Linear(3136, 40)
        self.fc2 = nn.Linear(40, 1)

        self.add_to_state(["conv1", "fc1", "fc2"])

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, 3136)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

plan = Net()
    
data_shape = (1, 3, 32, 32)
data = th.zeros(data_shape)
plan.build(data)
print(plan(data))
    
sent_plan = plan.send(alice)

tensor([[-0.1469]], grad_fn=<AddmmBackward>)


In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

gr.connect_all_nodes([bob, alice, charlie, dan])

In [11]:
# Fetch plan
fetched_plan = alice.fetch_plan("convnet", model_owner=me)
data_shape = (1, 3, 32, 32)
data = th.zeros(data_shape)
x_ptr = data.fix_prec().share(bob, charlie, crypto_provider=dan)

# TODO: this should be stored automatically
me._objects[x_ptr.id] = x_ptr

In [12]:
me._objects

AttributeError: 'Tensor' object has no attribute 'child'

In [5]:
# TODO: this should be done internally
new_state_ids = []
for state_id in fetched_plan.state_ids:
    a_sh = me._objects[state_id].fix_prec().share(bob, charlie, crypto_provider=dan).get()
    # TODO: this should be stored automatically
    me._objects[a_sh.id] = a_sh
    new_state_ids.append(a_sh.id)

In [6]:
fetched_plan.state_ids

[89202407203, 56780661394, 54079972075, 63518858733, 6715421912, 68929474864]

In [7]:
fetched_plan.replace_ids(fetched_plan.state_ids, new_state_ids)
fetched_plan.state_ids = new_state_ids

In [ ]:
me._objects

In [8]:
%%time
print(fetched_plan(x_ptr).get().float_prec())

tensor([[0.1420]])
CPU times: user 15.4 s, sys: 5.39 s, total: 20.8 s
Wall time: 18.6 s


In [8]:
# Support fetching a plan
plan_func = False

if plan_func:
    @sy.func2plan(args_shape=[(1,)], state={"bias": th.tensor([3.0])})
    def plan_mult_3(x, state):
        bias = state.read("bias")
        x = x * bias
        return x
else:
    class Net(sy.Plan):
        def __init__(self):
            super(Net, self).__init__(id="net2")
            self.fc1 = nn.Linear(1, 1)
            self.add_to_state(["fc1"])

        def forward(self, x):
            return self.fc1(x)
    
    plan_mult_3 = Net()
    plan_mult_3.build(th.tensor(1))

sent_plan = plan_mult_3.send(alice)
print(sent_plan.id)

net2


In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

gr.connect_all_nodes([bob, alice, charlie, dan])

In [9]:
# Fetch plan
fetched_plan = alice.fetch_plan("net2")

In [10]:
# TODO: this should be done internally
new_state_ids = []
for state_id in fetched_plan.state_ids:
    # TODO: we should not have direct access to the weights
    a_sh = me._objects[state_id].get()
    # TODO: this should be stored automatically
    me._objects[a_sh.id] = a_sh
    new_state_ids.append(a_sh.id)

In [11]:
fetched_plan.replace_ids(fetched_plan.state_ids, new_state_ids)
fetched_plan.state_ids = new_state_ids

In [12]:
x = th.tensor([1.])
print(fetched_plan(x))

tensor([0.2805], requires_grad=True)
